In [1]:
import pandas as pd

df = pd.read_csv("test.csv")

In [2]:
def filter_df(df):
    columns_to_drop = [
       'WT01', 'WT03', 'PRCP', 'SNOW', 'SNWD', 'TOBS', 'datacoverage',
       'mindate', 'maxdate', 'name', 'Unnamed: 0',
       'WT05', 'WT11', 'DAPR', 'MDPR', 'WESF', 'WT06', 'WT04', 'EVAP', 'MNPN',
       'MXPN', 'WDMV', 'SN32', 'SX32', 'WESD', 'SN52', 'SX52', 'SN31', 'SX31',
       'AWND', 'WDFG', 'WSFG', 'TAVG', 'PSUN', 'TSUN', 'WDF2', 'WDF5', 'WSF2',
       'WSF5', 'WT02', 'WT08', 'WT10', 'WT09', 'PGTM', 'MDSF', 'THIC', 'SN02', 'SN12', 'SX02',
       'SX12', 'DASF', 'DAEV', 'MDEV', 'WT07', 'FMTM', 'WT13', 'WT14', 'WT15',
       'WT16', 'WT17', 'WT18', 'WT19', 'WT21', 'WT22'
    ]
    df = df.drop(labels=columns_to_drop, axis="columns")
    return df[df["TMAX"].notnull()]

print(df.shape)

df["date"] = pd.to_datetime(df["date"])
df = filter_df(df)
df = df.sort_values(by=["date", "station"])

print(df.shape)

(511281, 73)
(485881, 9)


In [9]:
df.head()

,index,station,date,TMAX,TMIN,elevation,latitude,elevationUnit,longitude
0,0,GHCND:USC00140010,2009-01-01,7.2,-3.3,361.8,38.92660,METERS,-97.21290
122,0,GHCND:USC00140200,2009-01-01,2.2,-11.7,521.2,39.44830,METERS,-98.97020
123,0,GHCND:USC00140365,2009-01-01,4.4,-10.6,600.5,37.19410,METERS,-99.76320
43,0,GHCND:USC00140405,2009-01-01,-1.1,-10.0,288.0,39.57550,METERS,-95.11080
20,0,GHCND:USC00140439,2009-01-01,5.0,-11.7,881.8,39.79957,METERS,-101.04213


In [3]:
import numpy as np

def days_to_weeks(station, df):
    rows, cols = df.shape
    latitude = df.iloc[0]["latitude"]
    longitude = df.iloc[0]["longitude"]
    tmax = df["TMAX"]
    tmin = df["TMIN"]
    dates = df["date"]
    if rows % 7 != 0:
        num_elem_add = 7 - (rows % 7)
        tmax = tmax.append(pd.Series([np.nan for _ in range(num_elem_add)]))
        tmin = tmin.append(pd.Series([np.nan for _ in range(num_elem_add)]))
        dates = dates.append(pd.Series([np.nan for _ in range(num_elem_add)]))
        rows = len(tmax)
    tmax = tmax.values
    tmin = tmin.values
    dates = dates.values
    num_rows = rows // 7
    tmax = tmax.reshape(num_rows, 7)
    tmin = tmin.reshape(num_rows, 7)
    dates = dates.reshape(num_rows, 7)
    tmax_df = pd.DataFrame(tmax, columns=[f"tmax day {i}" for i in range(1, 8)])
    tmin_df = pd.DataFrame(tmin, columns=[f"tmin day {i}" for i in range(1, 8)])
    dates_df = pd.DataFrame(dates, columns=[f"date {i}" for i in range(1, 8)])
    final_df = pd.concat([tmax_df, tmin_df, dates_df], axis=1, join="inner")
    final_df["station"] = station
    final_df["latitude"] = latitude
    final_df["longitude"] = longitude
    return final_df
    
stations = []
final_df = pd.DataFrame()
for station, tmp in df.groupby("station"):
    final_df = final_df.append(days_to_weeks(station, tmp), ignore_index=True)
final_df.tail()

,tmax day 1,tmax day 2,tmax day 3,tmax day 4,tmax day 5,tmax day 6,tmax day 7,tmin day 1,tmin day 2,tmin day 3,...,date 1,date 2,date 3,date 4,date 5,date 6,date 7,station,latitude,longitude
69497,9.4,9.4,9.4,2.2,10.0,13.3,3.9,-6.6,-9.9,-8.2,...,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,GHCND:USW00093997,38.87611,-98.80917
69498,-1.6,-3.8,-2.1,5.0,1.7,3.3,-3.2,-9.3,-13.8,-13.2,...,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,GHCND:USW00093997,38.87611,-98.80917
69499,-2.1,-1.6,0.6,-4.3,-8.2,0.0,10.6,-12.1,-8.2,-5.5,...,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,GHCND:USW00093997,38.87611,-98.80917
69500,12.2,6.1,10.6,11.1,17.2,7.2,8.9,-4.3,-4.9,-4.9,...,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,GHCND:USW00093997,38.87611,-98.80917
69501,12.8,9.4,12.8,6.7,NaN,NaN,NaN,-3.8,-5.5,-6.6,...,2016-12-28,2016-12-29,2016-12-30,2016-12-31,NaT,NaT,NaT,GHCND:USW00093997,38.87611,-98.80917


In [4]:
import networkx as nx

def power(x):
    return pow(x, 2)

def sqrt(x):
    return pow(x, 0.5)

def distance_function(vect_one, vect_two):
    cols = {col: col.replace("_x", "") for col in vect_one.columns.tolist()}
    vect_one.rename(columns=cols, inplace=True)
    cols = {col: col.replace("_y", "") for col in vect_two.columns.tolist()}
    vect_two.rename(columns=cols, inplace=True)
    diffs = [
        vect_one[col] - vect_two[col]
        for col in vect_one.columns
    ]
    sq_diffs = [power(diff) for diff in diffs]
    return sqrt(sum(sq_diffs))
    
def mean(x):
    return sum(x)/len(x)
        
def dates_joined(station_one_df, station_two_df):
    dates = [f"date {i}" for i in range(1, 8)]
    joined = []
    for date_i in dates:
        for date_j in dates:
            joined.append(pd.merge(
                station_one_df, station_two_df,
                how="inner", left_on=date_i, 
                right_on=date_j
            ))
    return joined

def select_joined(joined):
    joined = sorted(
        [(join.shape[0], join) for join in joined],
        key=lambda t: t[0]
    )
    return joined[-1][1]
        
def clip_dates(joined_data):
    cols = joined_data.columns.tolist()
    cols = [col for col in cols if "_y" not in col and "date" not in col]
    date_cols = {col:col.replace("_x", "") for col in cols if "_x" in col and "date" in col}
    joined_data = joined_data[cols]
    joined_data = joined_data.rename(columns=date_cols)
    return joined_data
    
def calculate_average_weekly_temp_distance(station_one_df, station_two_df):
    joined = dates_joined(station_one_df, station_two_df)
    joined_data = select_joined(joined)
    #joined_data = clip_dates(joined_data)
    one_tmax_cols = [f"tmax day {i}_x" for i in range(1, 8)]
    two_tmax_cols = [f"tmax day {i}_y" for i in range(1, 8)]
    one_tmax_df = joined_data[one_tmax_cols]
    two_tmax_df = joined_data[two_tmax_cols]
    tmax_distances = distance_function(one_tmax_df, two_tmax_df)

    one_tmin_cols = [f"tmin day {i}_x" for i in range(1, 8)]
    two_tmin_cols = [f"tmin day {i}_y" for i in range(1, 8)]
    one_tmin_df = joined_data[one_tmin_cols]
    two_tmin_df = joined_data[two_tmin_cols]
    
    
#     one_df, two_df = align_data_based_on_dates(station_one_df, station_two_df)
#     tmax_cols = [f"tmax day {i}" for i in range(1, 8)]
#     tmin_cols = [f"tmin day {i}" for i in range(1, 8)]
#     one_tmax_df = one_df[tmax_cols]
#     two_tmax_df = two_df[tmax_cols]
#     tmax_distances = distance_function(one_tmax_df, two_tmax_df)
#     one_tmin_df = one_df[tmin_cols]
#     two_tmin_df = two_df[tmin_cols]
    tmin_distances = distance_function(one_tmin_df, two_tmin_df)
    tmax_distances.fillna(0, inplace=True)
    tmin_distances.fillna(0, inplace=True)
    return mean([mean(tmax_distances),  mean(tmin_distances)])
    
# set up graph
def gen_graph():
    G = nx.Graph()
    [G.add_node(station) for station in final_df["station"].unique()]
    count = 0
    for station_i in final_df["station"].unique():
        for station_j in final_df["station"].unique():
            if station_i == station_j:
                continue
            dates = [f"date {i}" for i in range(1, 8)]
            station_one_df = final_df[final_df["station"] == station_i]
            station_two_df = final_df[final_df["station"] == station_j]
            try:
                average_weekly_temp = calculate_average_weekly_temp_distance(
                    station_one_df, station_two_df
                )
                G.add_edge(station_i, station_j, average_weekly_temp_distance=average_weekly_temp)
                #print(average_weekly_temp)
                count += 1
            except:
                print("station_i", station_one_df.shape)
                print("station_j", station_two_df.shape)
            if count == 1000:
                return G
        
G = gen_graph()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


station_i (22, 24)
station_j (34, 24)
station_i (22, 24)
station_j (355, 24)
station_i (22, 24)
station_j (348, 24)
